In [1]:
%%capture
%pip install llama-index mistralai llama-index-embeddings-mistralai qdrant-client llama-index-vector-stores-qdrant llama-index-llms-mistralai

In [2]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [3]:
MISTRAL_API_KEY = os.environ['MISTRAL_API_KEY']
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [4]:
from llama_index.core.settings import Settings
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding


Settings.llm = MistralAI(
    model="mistral-small-latest", 
    api_key=MISTRAL_API_KEY
)


Settings.embed_model = MistralAIEmbedding(
    model_name="mistral-embed", 
    api_key=MISTRAL_API_KEY
)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/llama/lib/python3.13/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


# Recap of the LlamaIndex Order of Operations

In LlamaIndex, the order of operations in the query pipeline typically follows these steps:

**🍽️ Data Ingestion:** This is where your existing data from various sources and formats (APIs, PDFs, SQL, etc.) is ingested into the system.

**🗂️ Data Indexing:** The ingested data is structured into intermediate representations that are easy and performant for Large Language Models (LLMs) to consume.

**🐕 Retrieval:** Information is retrieved from your data sources based on the question or prompt. This is the first step in the Retrieval-Augmented Generation (RAG) process.

**🎖️ Reranking:** The initially retrieved documents or nodes are reordered based on certain criteria to bring the most relevant or useful nodes to the top.

**䷾ Post-processing:** After retrieval and reranking, transformations or filters are applied to the set of nodes to further refine them before they are used to generate the final response.

**💬 Response Generation:** The LLM generates a response based on the enriched prompt, which now includes the context from the retrieved and reranked documents.

# Data Ingestion

## 📥 Load the data

To use data with an LLM, first load it using data connectors, known as `Readers` in LlamaIndex, which format data into `Document` objects containing data and metadata.

📚 **SimpleDirectoryReader**:
  - The most straightforward loader is `SimpleDirectoryReader``.
  - Built into LlamaIndex, it reads various formats (Markdown, PDFs, Word documents, PowerPoint decks, images, audio, video) from every file in a directory, creating documents.

## 🔧 Transform the data

After loading, we must process and transform data for retrieval. We need to transform the list of `Document` objects into `Node` objects 

- ✂️ Include chunking, extracting metadata, and embedding each chunk in transformations.

- 🌟 Nodes are a first-class citizen in LlamaIndex, allowing direct definition or parsing from Documents.

- 🔄 Transformation inputs and outputs are `Node` objects (Note: `Document` is subclass of `Node`).

- 🛠️ Nodes are "chunks" of Documents, including text, images, etc., plus metadata and relationships.

- 📊 `NodeParser` classes convert Documents into Nodes with all necessary attributes. There are [a number of](https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules.html) `NodeParser`'s you can choose from!

- 📑 High-level API: Use `.from_documents()` for automatic parsing and chunking of Document objects.

- 🔍 Underlying process splits Document into Node objects, maintaining text and metadata with a link to their parent Document.

# Data Indexing

## 🗂️ Indexing

An `Index` is a data structure that allows for the quick retrieval of relevant context for a user query. 

It is the core foundation for retrieval-augmented generation (RAG) use-cases. Indexes are built from `Documents` and are used to build Retrievers, Query Engines and Chat Engines. All of which enable question & answer and chat over your data.

- 📂 After loading your data, you're ready to construct an `Index`.

- 🌐 **Vector Store Index:** The most common Index type. It segments your `Documents` into `Nodes` and generates vector embeddings for each node's text, prepping them for LLM queries.

- 🔄 **Vector Store Index Process:** Parse raw texts into document objects, split document objects into chunks/nodes, then convert all your nodes into embeddings and store them in a vector database.

## 🗄️ Storing

Loading and indexing data costs time and money.

By default, indexed data is stored in memory. But, you can store your data to avoid the time and costs associated with re-indexing them.  The simplest way to do this **persisting to disk**.

Each `Index` object has a `.persist()` method, which will write all the data to disk at the specified location.

### ☁️ Loading in a Vector Database

We'll use qdrant as our vector database of choice throughout this course.

To use qdrant to store embeddings from the `VectorStoreIndex`, you need to:

- Initialize the qdrant client

- Create a `Collection` to store your data in qdrant

- Assign qdrant as the `vector_store` in a `StorageContext`

- Initialize your `VectorStoreIndex` using that `StorageContext`

Below, we initialize a `QdrantClient` for interacting with qdrant, an open-source vector store.


### 🗃️ Storage Context

`StorageContext` in `LlamaIndex` is a core abstraction that revolves around the storage of `Nodes`, indices, and vectors.  It facilitates data storage and retrieval.

It is a utility container that supports the following:

 - `docstore`: A [`BaseDocumentStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/storage/docstore/types.py) for storing nodes.

 - `index_store`: A [`BaseIndexStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/storage/index_store/types.py#L13) for storing indices.

 - `vector_store`: A [`VectorStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/vector_stores/simple.py) for storing vectors.

 - `graph_store`: A [`GraphStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/graph_stores/simple.py) for storing knowledge graphs.

Below we instantiate the `StorageContext` from default settings indicating that we want to use a vector store.

# 🪃 Retrieval

A `Retriever` is an interface exposed by the `Index`. An `Index` with its `Retriever` is used for storing and fetching data. The `Retriever` is a part of the `Index` and is used to retrieve the data stored in the Index.


### LlamaIndex provides [many different types of retrievers](https://github.com/run-llama/llama_index/tree/main/llama-index-core/llama_index/core/retrievers) to fetch relevant information from ingested data based on a given query. 

Some examples include

### Vector Retriever

The vector retriever uses vector similarity search to find the most relevant nodes (chunks of text) based on the query embedding. It requires a vector database like to store and search through the node embeddings.

### [Fusion Retriever](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/fusion_retriever.py)

The fusion retriever generates multiple queries from the original query, performs retrieval over an ensemble of retrievers for each query, and then fuses and reranks the results across all queries. This aims to better capture the query intent through query rewriting and ensembling.

### [Recursive Retriever](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/recursive_retriever.py)

The recursive retriever allows for hierarchical retrieval by first retrieving coarse nodes and then recursively retrieving finer-grained nodes within those coarse nodes. This can be useful for multi-level indexing and retrieval.

You can also combine retrievers in interesting ways and build out more advanced retrieval strategies, as we will see later in this course.


### In the example here, we're using a Vector Retriever

 - 🔍 When searching, your query is also converted into a vector embedding. 
 
- 🗂️ The `VectorStoreIndex` then performs a mathematical operation to rank embeddings based on semantic similarity to your query.

- 🔝 Top-k semantic retrieval is the simplest wasy to query a vector index.

- ⩬ You can also apply a similarity threshold  (e.g., only return results that are more similar than some value)

# Reranking

# Post-processing

# Response Generation